<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 1 - Phase 3 - Renata

## Prerequisites

Make sure the prerequisites in [CL_LMDA_prerequisites](https://github.com/laelgelc/laelgelc/blob/main/CL_LMDA_prerequisites.ipynb) are satisfied.

## Dataset

Please download the following dataset (Right-click on the link and choose `Open link as a new tab` to download the corresponding file):
- [cl_st1_renata_truthbrush_20240616_raw_data_prep.jsonl](https://pucsp-my.sharepoint.com/:u:/g/personal/ra00341729_pucsp_edu_br/EcoAPWHvUI5GoB2g3hK_b60BlnN2Yo9duPY3xQu9EJRVpg?e=bkOelA)

## Importing the required libraries

In [1]:
import pandas as pd
import demoji
import re
import os
from collections import Counter
import html

## Data wrangling

### Importing the tweet raw data into a dataframe

In [2]:
df_tweets_raw_data = pd.read_json('cl_st1_renata_truthbrush_20240616_raw_data_prep.jsonl', lines=True)

In [3]:
df_tweets_raw_data.head(5)

,created_at,author_id,username,tweet_url,content,text
0,2024-06-16 21:27:19.392,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262841581...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Sunday Live: Hollywood Comes to...
1,2024-06-16 18:00:57.062,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262760432...,<p>Watch: Alex Jones Calls Out Globalist Tyran...,Watch: Alex Jones Calls Out Globalist Tyrants ...
2,2024-06-15 15:02:15.900,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11262123939...,<p>Alex Jones Emergency Saturday Broadcast: Le...,Alex Jones Emergency Saturday Broadcast: Learn...
3,2024-06-15 00:08:13.055,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261772385...,<p>Exclusive: Alex Jones Makes First Statement...,Exclusive: Alex Jones Makes First Statements A...
4,2024-06-14 17:19:17.792,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11261611591...,"<p><a href=""https://truthsocial.com/tags/AlexJ...","#AlexJonesShow LIVE: Tucker Carlson, Russell B..."


In [4]:
df_tweets_raw_data.shape

(762993, 6)

#### Inspecting a few tweets

In [5]:
inspected_row = 14
print('username:' + df_tweets_raw_data.loc[inspected_row, 'username'])
print('text:' + df_tweets_raw_data.loc[inspected_row, 'text'])
print('tweet_url:' + df_tweets_raw_data.loc[inspected_row, 'tweet_url'])

username:AlexJones
text:RT @infowarsGET READY: Tucker Carlson LIVE Nationwide Tour — Coming This Fall! LEARN MORE  https://www.infowars.com/posts/get-ready-tucker-carlson-live-nationwide-tour-coming-this-fall/
tweet_url:https://truthsocial.com/@AlexJones/112604587244214528


#### Exporting the raw data into a file for inspection

In [6]:
df_tweets_raw_data[['tweet_url', 'text']].to_csv('tweets_raw_data_url_text1.tsv', sep='\t', index=False)

In [7]:
df_tweets_raw_data[['text']].to_csv('tweets_raw_data_text1.tsv', sep='\t', index=False)

### Inspecting the dataset and eliminating malformed data

#### Identifying rows that are empty in column `text`

In [8]:
print(df_tweets_raw_data['text'].isnull().sum())

0


In [9]:
df_tweets_raw_data[df_tweets_raw_data['text'].isnull()]

,created_at,author_id,username,tweet_url,content,text


#### Dropping the rows that are empty in the column `text`

In [10]:
# Drop the rows whose column 'text' is NaN
df_tweets_raw_data = df_tweets_raw_data.dropna(subset=['text'])

# Reset the index
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

In [11]:
print(df_tweets_raw_data['text'].isnull().sum())

0


#### Checking if data types are consistent

In [12]:
df_tweets_raw_data.dtypes

created_at    datetime64[ns]
author_id              int64
username              object
tweet_url             object
content               object
text                  object
dtype: object

## Sampling the raw data according to search terms

In [13]:
# Defining the search terms
filter_words = ['gop', 'patriot', 'maga', 'awakening', 'storm', 'conservative', 'children', 'freedom', 'free speech']

# Creating a boolean mask for filtering
mask = df_tweets_raw_data['text'].str.contains('|'.join(filter_words), case=False)

# Applying the mask to create 'df_tweets_filtered'
df_tweets_filtered = df_tweets_raw_data[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(94295, 6)

In [14]:
df_tweets_filtered

,created_at,author_id,username,tweet_url,content,text
0,2024-05-17 20:09:57.926,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11245824232...,"<p>RT <span class=""h-card""><a href=""https://tr...",RT @infowarsTikTok Bans MUST-SEE Video Predict...
1,2024-05-17 16:19:17.494,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11245733527...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Friday LIVE: Weather Weapons Ma...
2,2024-05-08 16:14:16.500,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11240635475...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Must-Watch Wednesday Broadcast:...
3,2024-04-23 22:59:14.840,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11232301251...,<p>BREAKING: Australian Senator Calls For Elon...,BREAKING: Australian Senator Calls For Elon Mu...
4,2024-04-17 16:27:45.771,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11228749927...,"<p>RT <span class=""h-card""><a href=""https://tr...",RT @infowarsWatch: GOP Lawmakers Grill Mayorka...
...,...,...,...,...,...,...
94290,2022-05-08 16:00:00.486,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108267149...,"<p>RT <span class=""h-card""><a href=""https://tr...",RT @MelaniaTrump#HappyMothersDay to all the mo...
94291,2022-05-07 19:28:26.444,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108262307...,<p>BIG NEWS! Our inaugural Truth Social Patrio...,BIG NEWS! Our inaugural Truth Social Patriot h...
94292,2022-04-01 18:29:44.910,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108058233...,"<p>RT <span class=""h-card""><a href=""https://tr...",RT @dbonginoBig tech censorship is real. Read ...
94293,2022-03-11 15:08:04.927,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/107938531...,"<p>User engagement on <a href=""https://truthso...",User engagement on #TRUTH is off the charts…an...


## Cleaning data

### Removing specific Unicode characters

The dataset may need to be cleaned of invisible Unicode characters.

#### Detecting `U+2066`, `U+2069` and `U+FEFF` characters

- [U+2066](https://www.compart.com/en/unicode/U+2066)
- [U+2069](https://www.compart.com/en/unicode/U+2069)
- [U+FEFF](https://www.compart.com/en/unicode/U+FEFF)

Please refer to:
- [Python RegEx](https://www.w3schools.com/python/python_regex.asp)
- [regex101](https://regex101.com/)
- [RegExr](https://regexr.com/)

In [15]:
# Defining a function to detect specific Unicode characters
def extract_unicode_characters(df, column_name):
    unicode_chars = Counter()  # Initialize a Counter to store Unicode character counts

    for value in df[column_name]:
        if isinstance(value, str):
            # Use RegEx to find non-ASCII characters (Unicode)
#            non_ascii_chars = re.findall(r'[^\x00-\x7F]+', value)
            # Use RegEx to find specific Unicode characters - adjust the expression accordingly
            specific_unicode_chars = re.findall(r'[\u2066\u2069\ufeff]', value)
            unicode_chars.update(specific_unicode_chars)

    return unicode_chars

# Inspect the dataframe for specific Unicode characters
unicode_counts = extract_unicode_characters(df_tweets_filtered, 'text')

# Print the results
for char, count in unicode_counts.items():
    print(f'Character {char}: Count = {count}')

Character ﻿: Count = 209
Character ⁦: Count = 29
Character ⁩: Count = 20


#### Removing `U+2066`, `U+2069` and `U+FEFF` characters

In [16]:
# Defining a function to remove specific Unicode characters
def remove_specific_unicode(input_line):
    # Using RegEx to replace specific Unicode characters - adjust the expression accordingly
    cleaned_line = re.sub(r'[\u2066\u2069\ufeff]', '', input_line)
    return cleaned_line

# Removing specific Unicode characters
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(remove_specific_unicode)

### Replacing the `LF` character by a space

Some tweets, especially the retweeted ones, contain multiple lines of text.

In [17]:
# Defining a function to replace the `LF` character by a space
def remove_cr_lf(input_line):
    # Using RegEx to replace LF by a space
    cleaned_line = re.sub(r'\n', ' ', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(remove_cr_lf)

### Replacing the `U+00A0` (No-Break Space) character by a space

In [18]:
# Defining a function to replace the `U+A0` character by a space
def replace_no_break_space(input_line):
    # Using RegEx to replace LF by a space
    cleaned_line = re.sub(r'\u00a0', ' ', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_no_break_space)

### Removing URLs

In [19]:
# Defining a function to remove URLs
def remove_urls(input_string):
    modified_string = re.sub(r"((http|https):\/\/)?([a-zA-Z0-9-]+\.)+[a-zA-Z]{2,6}(\/[a-zA-Z0-9-._~:\/?#[\]@!$&'()*+,;=]*)?\/?", '', input_string)
    return modified_string

# Removing URLs
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(remove_urls)

### Cleaning HTML entities

In [20]:
def clean_html_entities(input_line):
    # Converting HTML entities to their corresponding characters
    decoded_line = html.unescape(input_line)
    # Removing HTML tags
    cleaned_line = re.sub(r'<.*?>', '', decoded_line)
    cleaned_line = re.sub(r'<', '', cleaned_line)
    cleaned_line = re.sub(r'>', '', cleaned_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(clean_html_entities)

### Removing the preceding `.`

In [21]:
# Defining a function to remove preceding the dot
def clean_text(input_line):
    # Removing preceding dot
    cleaned_line = re.sub(r'(^\.)(?=[ #\w+@])', '', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(clean_text)

### Dropping duplicates

#### Retweets

Retweets bear the following RegEx patterns at the beginning of the column `text`.

- ^RT @\w+\s*:
- ^rt @\w+\s*:
- ^RT @\w+\s*
- ^RT:\s*
- ^RT\s*

In [22]:
# Creating a boolean mask for filtering - it is preceded by '~' to invert the selection
mask = ~df_tweets_filtered['text'].str.contains(r'^RT @\w+\s*:|^rt @\w+\s*:|^RT @\w+\s*|^RT:\s*|^RT\s*', regex=True)

# Applying the mask to overwrite the raw data dataframe with non retweeted tweets
df_tweets_filtered = df_tweets_filtered[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(71739, 6)

#### Duplicate tweets

The dataset was build in a way that if a certain tweet had more than one photo, one copy of the tweet was included per unique photo. Since we are concerned with analysing just the text, those duplicates should be removed. Tweets that bear the same 'tweet_url' are duplicates - we are going to keep only the first.

In [23]:
df_tweets_filtered.drop_duplicates(subset='tweet_url', keep='first', inplace=True)
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(71739, 6)

#### @mentioned tweets

A few users @mention copies of tweets towards other specific users creating multiple copies of the same tweet - those duplicates should be removed.

In [24]:
# Create a new column 'no_mention' containing the contents of the column 'text' without any preceding @mentions
df_tweets_filtered['no_mention'] = df_tweets_filtered['text'].str.replace(r'^@\w+\s*', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_filtered.drop_duplicates(subset='no_mention', keep='first', inplace=True)
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(56646, 7)

#### Removing posts preceded by `'^\"\"'` pattern

For some reason, a few posts are empty: `""`. They should be removed.

In [25]:
# Creating a boolean mask for filtering - it is preceded by '~' to invert the selection
mask = ~df_tweets_filtered['text'].str.contains(r'^\"\"', regex=True)

# Applying the mask to overwrite the raw data dataframe with non retweeted tweets
df_tweets_filtered = df_tweets_filtered[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(56645, 7)

#### Duplicate texts

Checking for identical posts in terms of content of the column `text` in order to eliminate duplicates.

In [26]:
df_tweets_filtered.drop_duplicates(subset='text', keep='first', inplace=True)
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)
df_tweets_filtered.shape

(56645, 7)

## Inspecting and eliminating duplicates

### Creating a DataFrame index

In [27]:
df_tweets_filtered['df_index'] = df_tweets_filtered.index.astype(str).str.zfill(6)

### Sorting the DataFrame by the column `text` to enable duplicate detection

In [28]:
# Sorting the DataFrame by the 'text' column in ascending order
df_tweets_filtered = df_tweets_filtered.sort_values(by='text', ascending=True)

### Exporting the filtered data into a file for inspection

In [29]:
df_tweets_filtered[['df_index', 'text']].to_csv('tweets_emojified.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Inspecting a few tweets

In [30]:
inspected_row = 1277
print('text:' + df_tweets_filtered.loc[inspected_row, 'text'])

text:Dr. Peter McCullough thinks the Covid vaccine will play a big role in the 2024 election: “Whoever is running for president better figure this out pretty quick!”“25% of us didn’t take it (COVID vaccine) for a reason. The vaccine is kind of an important voter issue for us in health freedom. Another 15% are screwed by them. That’s 40% ... Whoever is running for president better figure this out pretty quick because only 60% of people vote.”


### Dropping identified duplicates

In [31]:
# Defining the list of indexes to drop
indexes_to_drop = [
    514, 
    52682
]

# Dropping the rows with the specified indexes
df_tweets_filtered = df_tweets_filtered.drop(indexes_to_drop)

### Sorting the DataFrame by the index to revert the DataFrame back to its original order

In [32]:
# Sorting the DataFrame back to the original order by the index
df_tweets_filtered = df_tweets_filtered.sort_index()
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)

In [33]:
df_tweets_filtered

,created_at,author_id,username,tweet_url,content,text,no_mention,df_index
0,2024-05-17 16:19:17.494,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11245733527...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Friday LIVE: Weather Weapons Ma...,#AlexJonesShow Friday LIVE: Weather Weapons Ma...,000000
1,2024-05-08 16:14:16.500,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11240635475...,"<p><a href=""https://truthsocial.com/tags/AlexJ...",#AlexJonesShow Must-Watch Wednesday Broadcast:...,#AlexJonesShow Must-Watch Wednesday Broadcast:...,000001
2,2024-04-23 22:59:14.840,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11232301251...,<p>BREAKING: Australian Senator Calls For Elon...,BREAKING: Australian Senator Calls For Elon Mu...,BREAKING: Australian Senator Calls For Elon Mu...,000002
3,2024-03-29 00:48:30.957,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11217622210...,<p>BREAKING: CPS Hands Over Kidnapped Children...,BREAKING: CPS Hands Over Kidnapped Children To...,BREAKING: CPS Hands Over Kidnapped Children To...,000003
4,2024-03-26 16:09:11.639,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11216285542...,<p>Tuesday LIVE: Vital US Port Brought to Stan...,Tuesday LIVE: Vital US Port Brought to Standst...,Tuesday LIVE: Vital US Port Brought to Standst...,000004
...,...,...,...,...,...,...,...,...
56638,2022-07-20 23:41:05.996,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108682311...,"<p>We are humbled by the welcome <a href=""http...",We are humbled by the welcome #TruthSocial is ...,We are humbled by the welcome #TruthSocial is ...,056640
56639,2022-05-09 21:43:42.330,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108274163...,<p>The wait is OVER! Don’t miss out on our ina...,The wait is OVER! Don’t miss out on our inaugu...,The wait is OVER! Don’t miss out on our inaugu...,056641
56640,2022-05-07 19:28:26.444,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108262307...,<p>BIG NEWS! Our inaugural Truth Social Patrio...,BIG NEWS! Our inaugural Truth Social Patriot h...,BIG NEWS! Our inaugural Truth Social Patriot h...,056642
56641,2022-03-11 15:08:04.927,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/107938531...,"<p>User engagement on <a href=""https://truthso...",User engagement on #TRUTH is off the charts…an...,User engagement on #TRUTH is off the charts…an...,056643


## Exporting to a file

### JSONL format

In [34]:
df_tweets_filtered[['created_at', 'author_id', 'username', 'tweet_url', 'text']].to_json('tweets_filtered.jsonl', orient='records', lines=True)

### TSV format

In [35]:
df_tweets_filtered[['created_at', 'author_id', 'username', 'tweet_url', 'text']].to_csv('tweets_filtered.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

## Importing the Target Corpus into a DataFrame

In [36]:
df_tweets_filtered = pd.read_json('tweets_filtered.jsonl', lines=True)

In [37]:
df_tweets_filtered.head(5)

,created_at,author_id,username,tweet_url,text
0,2024-05-17 16:19:17.494,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11245733527...,#AlexJonesShow Friday LIVE: Weather Weapons Ma...
1,2024-05-08 16:14:16.500,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11240635475...,#AlexJonesShow Must-Watch Wednesday Broadcast:...
2,2024-04-23 22:59:14.840,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11232301251...,BREAKING: Australian Senator Calls For Elon Mu...
3,2024-03-29 00:48:30.957,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11217622210...,BREAKING: CPS Hands Over Kidnapped Children To...
4,2024-03-26 16:09:11.639,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11216285542...,Tuesday LIVE: Vital US Port Brought to Standst...


In [38]:
df_tweets_filtered.shape

(56643, 5)

In [39]:
df_tweets_filtered.dtypes

created_at    datetime64[ns]
author_id              int64
username              object
tweet_url             object
text                  object
dtype: object

## Replacing hashtags

In [40]:
# Defining a function to format the hashtagged string
def format_hashtagged_string(input_line):
    # Defining a function to format the hashtagged string using RegEx
    def process_hashtagged_string(s):
            # Lowercase the string
            s = s.lower()
            # Add the appropriate prefixes and suffixes
            s = f'HASHTAG{s}_h'
            return s

    # Use RegEx to find and process each hashtagged string
    processed_line = re.sub(r'(#\w+)', lambda match: process_hashtagged_string(match.group(1)), input_line)
    return processed_line

# Formatting the hashtagged strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_hashtagged_string)

## Replacing emojis

### Demojifying the column `text`

In [41]:
# Defining a function to demojify a string
def demojify_line(input_line):
    demojified_line = demoji.replace_with_desc(input_line, sep='<em>')
    return demojified_line

df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(demojify_line)

#### Exporting the filtered data into a file for inspection

In [42]:
df_tweets_filtered[['text']].to_csv('tweets_demojified1_text.tsv', sep='\t', index=False)

### Separating the demojified strings with spaces

In [43]:
# Defining a function to separate the demojified strings with spaces
def preprocess_line(input_line):
    # Add a space before the first delimiter '<em>', if it is not already preceded by one
    preprocessed_line = re.sub(r'(?<! )<em>', ' <em>', input_line)
    # Add a space after the first delimiter '<em>', if it is not already followed by one
    preprocessed_line = re.sub(r'<em>(?! )', '<em> ', preprocessed_line)
    return preprocessed_line

# Separating the demojified strings with spaces
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(preprocess_line)

#### Exporting the filtered data into a file for inspection

In [44]:
df_tweets_filtered[['text']].to_csv('tweets_demojified2_text.tsv', sep='\t', index=False)

### Formatting the demojified strings

In [45]:
# Defining a function to format the demojified string
def format_demojified_string(input_line):
    # Defining a function to format the demojified string using RegEx
    def process_demojified_string(s):
            # Lowercase the string
            s = s.lower()
            # Replace spaces and colons followed by a space with underscores
            s = re.sub(r'(: )| ', '_', s)
            # Add the appropriate prefixes and suffixes
            s = f'EMOJI{s}e'
            return s

    # Use RegEx to find and process each demojified string
    processed_line = re.sub(r'<em>(.*?)<em>', lambda match: process_demojified_string(match.group(1)), input_line)
    return processed_line

# Formatting the demojified strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_demojified_string)

### Replacing the `pipe` character by the `-` character in the `text` column

Further on, a few columns of the dataframe are going to be exported into the file `tweets.txt` whose columns need to be delimited by the `pipe` character. Therefore, it is recommended that any occurrences of the `pipe` character in the `text` column are replaced by another character.

In [46]:
# Defining a function to replace the 'pipe' character by the '-' character
def replace_pipe_with_hyphen(input_string):
    modified_string = re.sub(r'\|', '-', input_string)
    return modified_string

# Replacing the 'pipe' character by the '-' character
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_pipe_with_hyphen)


#### Exporting the filtered data into a file for inspection

In [47]:
df_tweets_filtered[['text']].to_csv('tweets_demojified3_text.tsv', sep='\t', index=False)

## Tokenising

Please refer to [What is tokenization in NLP?](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/).

In [48]:
# Defining a function to tokenise a string
def tokenise_string(input_line):
    # Replace URLs with placeholders
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b'
    placeholder = '<URL>'  # Choose a unique placeholder
    urls = re.findall(url_pattern, input_line)
    tokenised_line = re.sub(url_pattern, placeholder, input_line)  # Replace URLs with placeholders
    
    # Replace curly quotes with straight ones
    tokenised_line = tokenised_line.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Separate common punctuation marks with spaces
    tokenised_line = re.sub(r'([.\!?,"\'/()])', r' \1 ', tokenised_line)
    # Add a space before '#'
    #tokenised_line = re.sub(r'(?<!\s)#', r' #', tokenised_line)  # Add a space before '#' if it is not already preceded by one
    # Reduce extra spaces by a single space
    tokenised_line = re.sub(r'\s+', ' ', tokenised_line)
    
    # Replace the placeholders with the respective URLs
    for url in urls:
        tokenised_line = tokenised_line.replace(placeholder, url, 1)
    
    return tokenised_line

# Tokenising the strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(tokenise_string)

## Creating the files `file_index.txt` and `tweets.txt`

### Creating column `text_id`

In [49]:
df_tweets_filtered['text_id'] = 't' + df_tweets_filtered.index.astype(str).str.zfill(6)

### Creating column `conversation`

In [50]:
df_tweets_filtered['author_id'] = df_tweets_filtered['author_id'].astype(str)

In [51]:
df_tweets_filtered['conversation'] = 'v:' + df_tweets_filtered['author_id'].str.replace('id_', '')

### Creating column `date`

In [52]:
# Convert 'created_at' to datetime format
df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])

# Extract the date part (without time) into a new column 'date'
df_tweets_filtered['date'] = df_tweets_filtered['created_at'].dt.date

# Add the prefix 'd:' to the 'date' values
df_tweets_filtered['date'] = 'd:' + df_tweets_filtered['date'].astype(str)

### Creating column `text_url`

In [53]:
df_tweets_filtered['text_url'] = 'url:' + df_tweets_filtered['tweet_url']

### Creating column `user`

In [54]:
df_tweets_filtered['user'] = 'u:' + df_tweets_filtered['username']

### Creating column `content`

In [55]:
df_tweets_filtered['content'] = 'c:' + df_tweets_filtered['text']

### Reordering the created columns

Please refer to:
- [Python - List Comprehension 1](https://www.w3schools.com/python/python_lists_comprehension.asp)
- [Python - List Comprehension 2](https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)

In [56]:
# Reorder the columns (we use list comprehension to create a list of all columns except 'text_id', 'variable', 'date' and 'text_url')
df_tweets_filtered = df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url', 'user', 'content'] + [col for col in df_tweets_filtered.columns if col not in ['text_id', 'conversation', 'date', 'text_url', 'user', 'content']]]

In [57]:
df_tweets_filtered

,text_id,conversation,date,text_url,user,content,created_at,author_id,username,tweet_url,text
0,t000000,v:107838014712814240,d:2024-05-17,url:https://truthsocial.com/@AlexJones/1124573...,u:AlexJones,c:HASHTAG#alexjonesshow_h Friday LIVE: Weather...,2024-05-17 16:19:17.494,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11245733527...,HASHTAG#alexjonesshow_h Friday LIVE: Weather W...
1,t000001,v:107838014712814240,d:2024-05-08,url:https://truthsocial.com/@AlexJones/1124063...,u:AlexJones,c:HASHTAG#alexjonesshow_h Must-Watch Wednesday...,2024-05-08 16:14:16.500,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11240635475...,HASHTAG#alexjonesshow_h Must-Watch Wednesday B...
2,t000002,v:107838014712814240,d:2024-04-23,url:https://truthsocial.com/@AlexJones/1123230...,u:AlexJones,c:BREAKING: Australian Senator Calls For Elon ...,2024-04-23 22:59:14.840,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11232301251...,BREAKING: Australian Senator Calls For Elon Mu...
3,t000003,v:107838014712814240,d:2024-03-29,url:https://truthsocial.com/@AlexJones/1121762...,u:AlexJones,c:BREAKING: CPS Hands Over Kidnapped Children ...,2024-03-29 00:48:30.957,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11217622210...,BREAKING: CPS Hands Over Kidnapped Children To...
4,t000004,v:107838014712814240,d:2024-03-26,url:https://truthsocial.com/@AlexJones/1121628...,u:AlexJones,c:Tuesday LIVE: Vital US Port Brought to Stand...,2024-03-26 16:09:11.639,107838014712814240,AlexJones,https://truthsocial.com/@AlexJones/11216285542...,Tuesday LIVE: Vital US Port Brought to Standst...
...,...,...,...,...,...,...,...,...,...,...,...
56638,t056638,v:107759501782461328,d:2022-07-20,url:https://truthsocial.com/@truthsocial/10868...,u:truthsocial,c:We are humbled by the welcome HASHTAG#truths...,2022-07-20 23:41:05.996,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108682311...,We are humbled by the welcome HASHTAG#truthsoc...
56639,t056639,v:107759501782461328,d:2022-05-09,url:https://truthsocial.com/@truthsocial/10827...,u:truthsocial,c:The wait is OVER ! Don ' t miss out on our i...,2022-05-09 21:43:42.330,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108274163...,The wait is OVER ! Don ' t miss out on our ina...
56640,t056640,v:107759501782461328,d:2022-05-07,url:https://truthsocial.com/@truthsocial/10826...,u:truthsocial,c:BIG NEWS ! Our inaugural Truth Social Patrio...,2022-05-07 19:28:26.444,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/108262307...,BIG NEWS ! Our inaugural Truth Social Patriot ...
56641,t056641,v:107759501782461328,d:2022-03-11,url:https://truthsocial.com/@truthsocial/10793...,u:truthsocial,c:User engagement on HASHTAG#truth_h is off th...,2022-03-11 15:08:04.927,107759501782461328,truthsocial,https://truthsocial.com/@truthsocial/107938531...,User engagement on HASHTAG#truth_h is off the ...


### Creating the file `file_index.txt`

In [58]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url']].to_csv('file_index.txt', sep=' ', index=False, header=False, encoding='utf-8', lineterminator='\n')

### Creating the file `tweets.txt`

In [59]:
folder = 'tweets'
try:
    os.mkdir(folder)
    print(f'Folder {folder} created!')
except FileExistsError:
    print(f'Folder {folder} already exists')

Folder tweets created!


Note: The parameters `doublequote=False` and `escapechar=' '` are required to avoid that the column content is doublequoted with '"' in sentences that use characters that need to be escaped such as double quote '"' itself - this causes a malformed response from TreeTagger.

In [60]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'user', 'content']].to_csv(f'{folder}/tweets.txt', sep='|', index=False, header=False, encoding='utf-8', lineterminator='\n', doublequote=False, escapechar=' ')

## Tagging with TreeTagger

- On Visual Studio Code (VS Code), open the folder where your project is located with `Open Folder...`
- Open a WSL Ubuntu Terminal on VS Code
- **Important**: Activate the `my_env` Python environment by executing `source "$HOME"/my_env/bin/activate`
- Proceed as indicated

Purpose: Annotate the texts in `tweets/tweets.txt` with part-of-speech and lemma information.
- Input
    - `file_index.txt`
    - `tweets/tweets.txt`
- Output
    - `tweets/tagged.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash treetagging.sh
--- treetagging t000000 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000001 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000002 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000003 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
<omitted>
```

## Processing `CL_St1_Ph21_Renata.ipynb`

Run the solution `CL_St1_Ph21_Renata.ipynb`

## Processing `tokenstypes`

Purpose: Capture the content tokens (specific occurrences of words) and the content types (general concept of words) from `tweets/tagged.txt`.
- Input
    - `file_index.txt`
    - `tweets/tagged.txt`
- Output
    - `tweets/tokens.txt`
    - `tweets/types.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash tokenstypes.sh
--- tokenstypes t000000 / 18206 ---
--- tokenstypes t000001 / 18206 ---
--- tokenstypes t000002 / 18206 ---
--- tokenstypes t000003 / 18206 ---
--- tokenstypes t000004 / 18206 ---
--- tokenstypes t000005 / 18206 ---
<omitted>
```

## Processing `toplemmas`

Purpose: Determine the 1.000 top lemmas. **Important**: This process requires manual inspection. Non-meaningful lemmas should be excluded by updating `stoplist.sed` and reiterating the processing. Proceed as indicated in this [video tutorial](https://youtu.be/4UIPpdoozP0?si=_3md9w79njZY86PE), from 20:05 to 25:16.
- Input
    - `tweets/types.txt`
    - `stoplist.sed`: List of rules that allows the exclusion of a certain lemmas
- Output
    - `selectedwords` = `var_index.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash toplemmas.sh
```

## Processing `sas`

Purpose: Prepare input data for processing in SAS.
- Input
    - `tweets/types.txt`
    - `selectedwords`
    - `file_index.txt`
- Output
    - `columns`
    - `sas/data.txt`
    - `sas/dates.txt`
    - `sas/wcount.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash sas.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
[nltk_data] Downloading package punkt to /home/eyamrog/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Word counts written to sas/wcount.txt
```

## Processing `datamatrix`

Purpose: Prepares input data for calculating the correlation matrix.
- Input
    - `file_index.txt`
    - `columns`
    - `selectedwords`
- Output
    - `file_ids.txt`
    - `data.csv`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash datamatrix.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
--- data.csv ...---
```

## Processing `correlationmatrix`

Purpose: Calculates the correlation matrix.
- Input
    - `data.csv`
- Output
    - `correlation`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash correlationmatrix.sh
--- python correlation ... ---
```

## Processing `formats`

Purpose: Prepare input data for processing in SAS.
- Input
    - `data.csv`
    - `selectedwords`
- Output
    - `sas/corr.txt`
    - `sas/word_labels_format.sas`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash formats.sh
--- sas/sas/corr.txt ---
--- sas/word_labels_format.sas ---
```

## Processing the statistical procedures on SAS

- Log in to your [SAS OnDemand for Academics](https://welcome.oda.sas.com/) account
- Proceed as indicated in this [video tutorial](https://youtu.be/I3u9zD3jyOA?si=68uIKVc2iusGG2KY)

## Processing `examples`

Purpose: Extract examples for analysis.
- Input
    - `sas/output_"$project"/loadtable.html`
    - `sas/output_"$project"/"$project"_scores.tsv`
    - `sas/output_"$project"/"$project"_scores_only.tsv`
- Output
    - `examples/factors`
    - `example files`

```
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash examples.sh
6780
1246
698
123
--- examples f1pos ---
--- factor 1 pos # 000001 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000002 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000003 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000004 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000005 ---
tr: warning: an unescaped backslash at end of string is not portable
<ommitted>
```